## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-07-17-49-17 +0000,nypost,Inside Jets superfan’s insane tailgate with ‘e...,https://nypost.com/2025/09/07/us-news/inside-j...
1,2025-09-07-17-47-57 +0000,nyt,"Trump Downplays Post Threatening Chicago, Sayi...",https://www.nytimes.com/2025/09/07/us/politics...
2,2025-09-07-17-46-03 +0000,nyt,"As Syria Tries to Move Away From Dictatorship,...",https://www.nytimes.com/2025/09/07/world/middl...
3,2025-09-07-17-43-29 +0000,nyt,Russia Strikes Ukraine Government Building in ...,https://www.nytimes.com/2025/09/07/world/europ...
4,2025-09-07-17-41-45 +0000,bbc,South Korea reaches deal to release citizens d...,https://www.bbc.com/news/articles/cj07jzgve45o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2368/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,16
33,ukraine,11
48,raid,10
31,russia,9
34,government,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
72,2025-09-07-12-19-07 +0000,nypost,South Korean workers arrested in ICE raid at G...,https://nypost.com/2025/09/07/us-news/south-ko...,79
48,2025-09-07-15-08-17 +0000,nypost,Russian attack on Ukraine’s government HQ in l...,https://nypost.com/2025/09/07/world-news/russi...,71
132,2025-09-07-03-00-00 +0000,wsj,When President Trump unveiled his EU trade dea...,https://www.wsj.com/economy/trade/the-eus-trad...,66
101,2025-09-07-09-34-43 +0000,cbc,Russia's largest air attack on Ukraine sets fi...,https://www.cbc.ca/news/world/russia-mass-assa...,62
32,2025-09-07-16-31-16 +0000,wapo,Russia’s largest-ever air attack on Ukraine bu...,https://www.washingtonpost.com/world/2025/09/0...,60


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
72,79,2025-09-07-12-19-07 +0000,nypost,South Korean workers arrested in ICE raid at G...,https://nypost.com/2025/09/07/us-news/south-ko...
101,62,2025-09-07-09-34-43 +0000,cbc,Russia's largest air attack on Ukraine sets fi...,https://www.cbc.ca/news/world/russia-mass-assa...
132,42,2025-09-07-03-00-00 +0000,wsj,When President Trump unveiled his EU trade dea...,https://www.wsj.com/economy/trade/the-eus-trad...
111,33,2025-09-07-08-42-01 +0000,nypost,Pope Leo XIV names first two saints of his pon...,https://nypost.com/2025/09/07/world-news/pope-...
71,32,2025-09-07-12-25-45 +0000,nyt,Prime Minister Shigeru Ishiba of Japan Says He...,https://www.nytimes.com/2025/09/07/world/asia/...
55,30,2025-09-07-14-00-00 +0000,wsj,The story of American capitalism may have begu...,https://www.wsj.com/economy/capitalism-america...
184,29,2025-09-06-18-44-36 +0000,bbc,Israel destroys second high-rise as assault on...,https://www.bbc.com/news/articles/cz9jpzql958o...
89,28,2025-09-07-10-48-17 +0000,nypost,Off-duty Alaska Airlines pilot who tried shutt...,https://nypost.com/2025/09/07/us-news/off-duty...
17,26,2025-09-07-17-09-14 +0000,nypost,"Undersea cables cut in Red Sea, interrupting i...",https://nypost.com/2025/09/07/world-news/under...
142,24,2025-09-07-01-00-00 +0000,wsj,"Trump has so far stood by RFK Jr., the health ...",https://www.wsj.com/politics/policy/trump-said...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
